# BoxCox.jl

A lightweight package with nice extensions

Phillip Alday  
2024-07-12

# The Box-Cox Transformation

## Box & Cox (1964)

$$
\begin{cases}
\frac{x^{\lambda} - 1}{\lambda} &\quad \lambda \neq 0 \\
\log x &\quad \lambda = 0
\end{cases}
$$

The denominator serves to normalize the transformation and preserve the
original direction of the effect (the sign is flipped when
$\lambda < 0$). In application, we may only care about the numerator
(e.g. when it suggests using “speed” instead of “time”.)

## Example: Square of a Normal Distribution

In [1]:
using BoxCox
using CairoMakie
using Random
CairoMakie.activate!(; type="svg")

x = abs2.(randn(MersenneTwister(42), 1000))
let f = Figure()
    ax = Axis(f[1,1]; xlabel="x", ylabel="density")
    density!(ax, x)
    ax = Axis(f[1,2]; xlabel="theoretical quantiles", ylabel="observed values")
    qqnorm!(ax, x)
    colsize!(f.layout, 1, Aspect(1, 1.0))
    colsize!(f.layout, 2, Aspect(1, 1.0))
    resize_to_layout!(f)
    f
end

## Fitting the transformation

In [1]:
bc = fit(BoxCoxTransformation, x)

Box-Cox transformation

estimated λ: 0.2073
resultant transformation:

 y^0.2 - 1
-----------
    0.2

## Examining the fitted transformation

> **Makie extension**
>
> If an appropriate Makie backend is loaded, then you can also do
> diagnostic plots.

In [1]:
let f = Figure(; size=(500, 300))
    ax = Axis(f[1,1])
    boxcoxplot!(ax, bc;
                conf_level=0.95)
    f
end

## Applying the fitted transformation

In [1]:
bc.(x)'

1×1000 adjoint(::Vector{Float64}) with eltype Float64:
 -1.04197  -1.37758  -3.74249  -1.89776  …  -3.3449  -0.295085  -1.06768

In [1]:
let f = Figure(; size=(800, 300)), bcx = bc.(x)
    ax = Axis(f[1,1]; xlabel="x", ylabel="density")
    density!(ax, bcx)
    ax = Axis(f[1,2]; xlabel="theoretical quantiles", ylabel="observed values")
    qqnorm!(ax, bcx; qqline=:fitrobust)
    colsize!(f.layout, 1, Aspect(1, 1.0))
    colsize!(f.layout, 2, Aspect(1, 1.0))
    resize_to_layout!(f)
    f
end

# Conditional Distributions: transforming the response of a regression model

## Example: Tree Growth

In [1]:
using DataFrames
using RDatasets: dataset as rdataset
trees = rdataset("datasets", "trees")

In [1]:
describe(trees)

R Core Team (2024); Meyer (1953)

## Linear Regression

``` julia
y = trees[!, :Volume]
X = hcat(ones(length(y)),
         log.(trees[!, :Height]),
         log.(trees[!, :Girth]))
bc_tree = fit(BoxCoxTransformation, X, y)
```

    Box-Cox transformation

    estimated λ: -0.0673
    resultant transformation:

     y^-0.1 - 1
    ------------
        -0.1

``` julia
let f = Figure(; size=(400, 300))
  boxcoxplot!(Axis(f[1,1]), bc_tree; conf_level=0.95)
  f
end
```



> **Diagnostics matter!**
>
> $\lambda=0$ is well within the 95% CI and log fits in well with the
> rest of the model.

## Linear Regression

In [1]:
using StatsModels
fit(BoxCoxTransformation,
    @formula(Volume ~ 1 +
                      log(Height) +
                      log(Girth)),
    trees)

Box-Cox transformation

estimated λ: -0.0673
resultant transformation:

 y^-0.1 - 1
------------
    -0.1

> **StatsModels extension**
>
> If StatsModels.jl is loaded (even indirectly via e.g. GLM.jl), then
> you can also use the `@formula` macro to specify the regression model.

# This also works for mixed models!

## Reaction time in the sleep study

In [1]:
using MixedModels
model = fit(MixedModel,
            @formula(reaction ~ 1 + days + (1 + days | subj)),
            dataset(:sleepstudy))

## Fitting the transformation

``` julia
bc_mixed = fit(BoxCoxTransformation,
               model)
```

    Box-Cox transformation

    estimated λ: -1.0747
    resultant transformation:

     y^-1.1 - 1
    ------------
        -1.1

``` julia
let f = Figure(; size=(400, 300))
  boxcoxplot!(Axis(f[1,1]), bc_mixed; conf_level=0.95)
  f
end
```



> **Diagnostics matter!**
>
> $\text{time}^{-1}$ has a natural interpretation as *speed* and -1 is
> nearly as good as the “optimal” transformation. We thus use our domain
> expertise and use *speed* (as responses per second) instead of the
> fitted result.

## Speed in the sleep study

In [1]:
model_bc = fit(MixedModel,
               @formula(1000 / reaction ~ 1 + days + (1 + days | subj)),
               dataset(:sleepstudy))

> **Note**
>
> We use 1000 in the numerator to scale things back to seconds from
> milliseconds.

# References

Box, G. E. P., & Cox, D. R. (1964). <span class="nocase">An Analysis of
Transformations</span>. *Journal of the Royal Statistical Society:
Series B (Methodological)*, *26*(2), 211–243.
<https://doi.org/10.1111/j.2517-6161.1964.tb00553.x>

Meyer, H. A. (1953). *Forest mensuration.* Penns Valley Publishers.

R Core Team. (2024). *R: A language and environment for statistical
computing*. R Foundation for Statistical Computing.
<https://www.R-project.org/>

*This page was rendered from git revision [155230d
](https://github.com/palday/juliacon2024/tree/155230d
).*